In [1]:
import os
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score, precision_recall_fscore_support

In [2]:
mysql_user = os.environ['MYSQL_USER']
mysql_password = os.environ['MYSQL_PASSWORD']
mysql_host = "remote_mysql:3306"
mysql_db = os.environ['MYSQL_DATABASE']

engine = create_engine(f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}")
conn = engine.connect()

In [52]:
query = 'SELECT token, ' \
        + 'STR_TO_DATE(CONCAT(trade_date, " ", trade_time), "%%Y-%%m-%%d %%H:%%i:%%s") AS trade_datetime, ' \
        + 'krw_balance, token_balance, trade_call, target ' \
        + 'FROM Trade_Log ' \
        + 'ORDER BY trade_datetime ASC ' \
        + 'LIMIT 1001;'
df = pd.read_sql(query, conn)

df['24h_accuracies'] = 0.0
df['24h_precision'] = 0.0
df['24h_recall'] = 0.0
df['24h_f1'] = 0.0

t1 = df.loc[len(df) - 1]
df = df.loc[:len(df) - 2]

In [35]:
query = 'SELECT token, ' \
        + 'STR_TO_DATE(CONCAT(trade_date, " ", trade_time), "%%Y-%%m-%%d %%H:%%i:%%s") AS trade_datetime, ' \
        + 'krw_balance, token_balance, trade_call, target ' \
        + 'FROM Trade_Log ' \
        + 'ORDER BY trade_datetime ASC ' \
        + 'LIMIT 1001;'
test_df = pd.read_sql(query, conn)

test_df['24h_accuracies'] = 0.0
test_df['24h_precision'] = 0.0
test_df['24h_recall'] = 0.0
test_df['24h_f1'] = 0.0

test_df = test_df.loc[1:].reset_index(drop = True)

In [53]:
### Get Initial Confusion Matrix and R_0 from Dataframe at T=0
### Get R_t+1 from Dataframe at T=1
### Update Confusion Matrix using R_0 and R_t+1
### Update R_0

def get_r0(df, labels):
    r0 = np.array([[[0, 0], [0, 0]]])
    for i in range(len(labels) - 1):
        r0 = np.concat((r0, np.array([[[0, 0], [0, 0]]])), axis = 0)

    ### First Record in Dataframe (R_0)
    pred = df.loc[0, 'trade_call']
    target = df.loc[0, 'target']
    for j, label in enumerate(labels):
        pred_pos = 1 if pred == label else 0
        actual_pos = 1 if target == label else 0
        r0[j, pred_pos, actual_pos] += 1
    
    return r0

def create_confusion_matrix(df, labels):
    ### Initialize an Empty Confusion Matrix per Class
    r0 = get_r0(df, labels)
    
    ### Confusion Matrix
    arr = r0.copy()
    for i in range(1, len(df)):
        pred = df.loc[i, 'trade_call']
        target = df.loc[i, 'target']

        for j, label in enumerate(labels):
            pred_pos = 1 if pred == label else 0
            actual_pos = 1 if target == label else 0
            arr[j, pred_pos, actual_pos] += 1
    
    return arr, r0, df.loc[1:].reset_index(drop = True)

def update_confusion_matrix(t1, r0, cm_t, df, labels):
    rt1 = np.array([[[0, 0], [0, 0]]])
    for i in range(len(labels) - 1):
        rt1 = np.concat((rt1, np.array([[[0, 0], [0, 0]]])), axis = 0)

    ## Last Record in New Dataframe (R_t+1)
    for j, label in enumerate(labels):
        pred_pos = 1 if t1['trade_call'] == label else 0
        actual_pos = 1 if t1['target'] == label else 0
        rt1[j, pred_pos, actual_pos] += 1
    
    cm_t1 = cm_t - r0 + rt1
    r0 = get_r0(df, labels)

    df = pd.concat([df, t1.to_frame().T], axis = 0)
    df = df.loc[1:].reset_index(drop = True)

    return cm_t1, r0, df

In [54]:
df

,token,trade_datetime,krw_balance,token_balance,trade_call,target,24h_accuracies,24h_precision,24h_recall,24h_f1
0,KRW-BTC,2025-01-21 19:55:15,783,0.000093,ask,hold,0.0,0.0,0.0,0.0
1,KRW-BTC,2025-01-21 19:56:16,15184,0.000000,bid,hold,0.0,0.0,0.0,0.0
2,KRW-BTC,2025-01-21 19:57:18,753,0.000093,hold,hold,0.0,0.0,0.0,0.0
3,KRW-BTC,2025-01-21 19:58:19,753,0.000093,hold,bid,0.0,0.0,0.0,0.0
4,KRW-BTC,2025-01-21 19:59:21,753,0.000093,bid,ask,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
995,KRW-BTC,2025-01-22 12:56:30,584,0.000071,ask,hold,0.0,0.0,0.0,0.0
996,KRW-BTC,2025-01-22 12:57:32,11770,0.000000,bid,hold,0.0,0.0,0.0,0.0
997,KRW-BTC,2025-01-22 12:58:33,584,0.000071,ask,hold,0.0,0.0,0.0,0.0
998,KRW-BTC,2025-01-22 12:59:35,11759,0.000000,ask,hold,0.0,0.0,0.0,0.0


In [7]:
multilabel_confusion_matrix(df['trade_call'], df['target'], labels = ['bid', 'hold', 'ask'])

array([[[420, 104],
        [383,  93]],

       [[335, 535],
        [ 69,  61]],

       [[473, 133],
        [320,  74]]])

In [56]:
t_precision, t_recall, t_f1, _ = precision_recall_fscore_support(df['target'], df['trade_call'], average = 'weighted', zero_division = 1.0)

In [58]:
def custom_precison_recall_fscore(confusion_matrix, labels, average = 'weighted', n_digits = 3):
    #############       Neg   TN: [0, 0]   FN: [0, 1]
    ############# Pred
    #############       Pos   FP: [1, 0]   TP: [1, 1]
    #############                 Neg         Pos
    #############                      Actual
    
    avg_precision, avg_recall, avg_f1 = 0, 0, 0

    if average == "weighted":
        supports = []
        precisions = []
        recalls = []
        f1_scores = []
        for i, label in enumerate(labels):
            ### True Positive + False Negative
            supports.append(confusion_matrix[i][0, 1] + confusion_matrix[i][1, 1])
            ### TP / (TP + FP)
            precision = confusion_matrix[i][1, 1] / (confusion_matrix[i][1, 1] + confusion_matrix[i][1, 0])
            
            ### TP / (TP + FN)
            recall = confusion_matrix[i][1, 1] / (confusion_matrix[i][1, 1] + confusion_matrix[i][0, 1])

            f1 = 2 * ((recall * precision) / (recall + precision))

            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
        
        weights = []
        for support in supports:
            weights.append(support / sum(supports))
    
        for i in range(len(labels)):
            avg_precision += precisions[i] * weights[i]
            avg_recall += recalls[i] * weights[i]
            avg_f1 += f1_scores[i] * weights[i]
    
    return round(float(avg_precision), n_digits), round(float(avg_recall), n_digits), round(float(avg_f1), n_digits)

In [59]:
labels = ['bid', 'hold', 'ask']
cm_t, r0, df = create_confusion_matrix(df, labels)

In [60]:
precision, recall, f1 = custom_precison_recall_fscore(cm_t, labels = labels)

In [61]:
precision, recall, f1

(0.357, 0.228, 0.206)

In [63]:
round(t_precision, 3), round(t_recall, 3), round(t_f1, 3)

(0.357, 0.228, 0.206)

In [ ]:
precision_recall_fscore_support

In [11]:
cm_t, r0, df = update_confusion_matrix(t1, r0, cm_t, df, labels)

In [32]:
cm_t

array([[[419, 104],
        [384,  93]],

       [[335, 535],
        [ 69,  61]],

       [[474, 133],
        [319,  74]]])

In [10]:
multilabel_confusion_matrix(test_df['trade_call'], test_df['target'], labels = labels)

array([[[419, 104],
        [384,  93]],

       [[335, 535],
        [ 69,  61]],

       [[474, 133],
        [319,  74]]])

In [2]:
for i in range(10):
    pass

In [4]:
for j in range(i, 20):
    print(j)

9
10
11
12
13
14
15
16
17
18
19


In [71]:
df[:10]

,token,trade_datetime,krw_balance,token_balance,trade_call,target,24h_accuracies,24h_precision,24h_recall,24h_f1
0,KRW-BTC,2025-01-21 19:56:16,15184,0.000000,bid,hold,0.0,0.0,0.0,0.0
1,KRW-BTC,2025-01-21 19:57:18,753,0.000093,hold,hold,0.0,0.0,0.0,0.0
2,KRW-BTC,2025-01-21 19:58:19,753,0.000093,hold,bid,0.0,0.0,0.0,0.0
3,KRW-BTC,2025-01-21 19:59:21,753,0.000093,bid,ask,0.0,0.0,0.0,0.0
4,KRW-BTC,2025-01-21 20:00:22,753,0.000093,bid,bid,0.0,0.0,0.0,0.0
5,KRW-BTC,2025-01-21 20:01:23,753,0.000093,hold,hold,0.0,0.0,0.0,0.0
6,KRW-BTC,2025-01-21 20:02:25,753,0.000093,hold,hold,0.0,0.0,0.0,0.0
7,KRW-BTC,2025-01-21 20:03:26,753,0.000093,bid,hold,0.0,0.0,0.0,0.0
8,KRW-BTC,2025-01-21 20:04:27,753,0.000093,bid,hold,0.0,0.0,0.0,0.0
9,KRW-BTC,2025-01-21 20:05:29,753,0.000093,ask,hold,0.0,0.0,0.0,0.0


In [65]:
i

9